Anomalies in Text Classification

In [2]:
import tensorflow as tf

**Import in-built datasets and tensorflow hub to use a pretrained model.**

In [3]:
import tensorflow_datasets as tfds
import tensorflow_hub as hub


**To see the list of all the built in databases in tensorflow.**

In [ ]:
#tfds.list_builders()

In [4]:
traindata,testdata =tfds.load(name='yelp_polarity_reviews',
          split=(tfds.Split.TRAIN,tfds.Split.TEST),
          as_supervised=True
          
          )

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/yelp_polarity_reviews/plain_text/0.1.0.incompleteNXHQ3G/yelp_polarity_reviews-train.tfrecord


  0%|          | 0/560000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/yelp_polarity_reviews/plain_text/0.1.0.incompleteNXHQ3G/yelp_polarity_reviews-test.tfrecord


  0%|          | 0/38000 [00:00<?, ? examples/s]

Dataset yelp_polarity_reviews downloaded and prepared to /root/tensorflow_datasets/yelp_polarity_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


In [ ]:
teb,tlb = next(iter(traindata.batch(10)))

In [ ]:
teb

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"The Groovy P. and I ventured to his old stomping grounds for lunch today.  The '5 and Diner' on 16th St and Colter left me with little to ask for.  Before coming here I had a preconceived notion that 5 & Diners were dirty and nasty. Not the case at all.\\n\\nWe walk in and let the waitress know we want to sit outside (since it's so nice and they had misters).  We get two different servers bringing us stuff (talk about service) and I ask the one waitress for recommendations.  I didn't listen to her, of course, and ordered the Southwestern Burger w/ coleslaw and started with a nice stack of rings.\\n\\nThe Onion Rings were perfectly cooked.  They looked like they were prepackaged, but they were very crispy and I could actually bite through the onion without pulling the entire thing out (don't you hate that?!!!)\\n\\nThe Southwestern Burger was order Medium Rare and was cooked accordingly.  Soft, juicy, and pink with a nice crispy bro

In [ ]:
tlb

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([1, 0, 0, 0, 1, 1, 1, 1, 1, 1])>

In [ ]:
pretrained_model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(pretrained_model, input_shape =[], dtype =tf.string, trainable =True)

In [ ]:
hub_layer(teb[:2])

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[ 3.1490684 , -3.6443894 ,  1.0309498 ,  2.6312134 , -8.113033  ,
        -4.344852  , -4.7093506 ,  2.8570368 ,  4.951268  , -0.01112661,
        -4.7766633 ,  4.3316603 , -0.39691204,  0.53871346, -7.611556  ,
         0.7667435 ,  5.3026733 , -1.3667821 , -4.735002  , -3.4333549 ],
       [ 0.7355903 , -0.98872495,  0.8112321 ,  0.88434154, -2.8713462 ,
        -2.2590046 , -1.1239421 ,  1.250997  ,  0.8693217 , -0.5148209 ,
        -1.1298319 ,  1.8178903 , -0.35861325, -0.36168805, -2.4337575 ,
         0.14544375,  3.0567162 ,  0.31669778, -1.0143214 , -1.4643673 ]],
      dtype=float32)>

In [ ]:
type(teb[:2])

NameError: ignored

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16,activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [1]:
model.fit(traindata.shuffle(10000).batch(512),
          epochs=20,
          verbose=1)

NameError: ignored

**Lets Check the Accuracy of the Model with a tricky review.**

In [ ]:
model.predict(["The food is kind of O.K but the ambience is not that bad, rather it exceeds expectations. The food serving time is below optimal"])

array([[0.04039075]], dtype=float32)

**Though the model works well with easy to comprehend keywords.**

In [ ]:
model.predict(["The food is super good but the ambience is excellent, rather it exceeds expectations. The serving time is great"])

array([[0.9540932]], dtype=float32)

In [ ]:
model.predict(["The food is just about tasty, the rooms are tidy yet very small. The room service is quick though it is expensive"])

array([[0.64542717]], dtype=float32)

In [ ]:
model.predict(["Food is disgusting. service is above average and ambience is noisy and fantastic"])

array([[0.10393415]], dtype=float32)

In [ ]:
model.predict(["I’ve got write this review after a compelling stay at the hotel. It was compelling in more than one ways. The hotel serves literally the food that you will not find elsewhere. It is short of ecstatic to hang out at the pool which was surprisingly huge with low fees. Rooms are tidy, cozy and up to the mark."])

array([[0.00443476]], dtype=float32)